# Pattern-centric Fossil Recognition (Geo Fossils-I)
- Preprocess + mask fossils, compute classical texture features (154-dim)
- Train ResNet-50 + ViT (ImageNet init), collect embeddings
- Hybrid classifier (texture + deep)
- XAI: Grad-CAM, occlusion; TCAV optional if concept patches provided

**Result figures produced** (saved under `figures/`):
- `cm_cnn.png`, `cm_vit.png`: confusion matrices
- `hybrid_f1.png`: macro-F1 comparison (texture/deep/hybrid)
- `gradcam_cnn.png`: Grad-CAM overlay highlighting texture cues
- `occlusion_cnn.png`: occlusion sensitivity heatmap
- `tcav_cnn.png` (optional): concept influence if `concept_bank/<concept>` patches exist

In [125]:
import argparse
import json
import math
import os
import random
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.auto import tqdm
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from captum.attr import Occlusion
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
from skimage.filters import gabor
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms.functional import to_pil_image

plt.switch_backend("Agg")  # headless plotting

@dataclass
class Config:
    root: Path
    img_size: int = 224
    batch_size: int = 1024
    num_workers: int = 0
    max_epochs: int = 100
    patience: int = 8
    lr: float = 1e-4
    weight_decay: float = 1e-4
    seeds: Tuple[int, ...] = (13, 23, 33)
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    figdir: Path = Path("figures")
    ckptdir: Path = Path("checkpoints")
    concept_root: Path = Path("concept_bank")

#### Utilities

In [126]:
def set_seed(seed: int) -> None: # set random seeds for reproducibility
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def ensure_dir(path: Path) -> None: # create directory if it doesn't exist
    path.mkdir(parents=True, exist_ok=True)

def list_images_by_class(root: Path) -> List[Dict]: # list image paths and labels
    manifest = []
    classes = sorted([d for d in root.iterdir() if d.is_dir()]) # get class directories
    class_to_idx = {cls.name: i for i, cls in enumerate(classes)} # map class names to indices
    for cls in classes:
        for img_path in sorted(cls.glob("*.jpg")):
            manifest.append({"path": img_path, "label": class_to_idx[cls.name], "classname": cls.name}) # add image info to manifest
    return manifest

def stratified_split(manifest: List[Dict], train_ratio=0.7, val_ratio=0.15) -> Dict[str, List[Dict]]: # stratified train/val/test split
    df = pd.DataFrame(manifest)
    train_df, temp_df = train_test_split(df, test_size=1 - train_ratio, stratify=df["label"], random_state=42) 
    rel_val = val_ratio / (1 - train_ratio)
    val_df, test_df = train_test_split(temp_df, test_size=1 - rel_val, stratify=temp_df["label"], random_state=99)
    return {"train": train_df.to_dict("records"), "val": val_df.to_dict("records"), "test": test_df.to_dict("records")}

def resize_and_pad(img: np.ndarray, size: int = 224) -> np.ndarray: # resize and pad image to square
    h, w = img.shape[:2]
    scale = size / min(h, w)
    new_h, new_w = int(round(h * scale)), int(round(w * scale))
    resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)
    pad_vert = max(size - new_h, 0)
    pad_h1 = pad_vert // 2
    pad_h2 = pad_vert - pad_h1
    pad_horiz = max(size - new_w, 0)
    pad_w1 = pad_horiz // 2
    pad_w2 = pad_horiz - pad_w1
    padded = cv2.copyMakeBorder(resized, pad_h1, pad_h2, pad_w1, pad_w2, cv2.BORDER_CONSTANT, value=0)
    return padded

def clean_mask(mask: np.ndarray, min_component: int = 50) -> np.ndarray:            # clean by removing small components
    mask = (mask > 0).astype(np.uint8)
    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)             # remove noise
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)            # close gaps
    num, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)  # connected components
    if num <= 1:
        return mask
    largest = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])                            # find largest component
    clean = (labels == largest).astype(np.uint8)                                    # retain largest component
    if stats[largest, cv2.CC_STAT_AREA] < min_component:                            # if too small, return empty mask
        return np.zeros_like(mask, dtype=np.uint8)                                  # empty mask
    return clean

def compute_mask(gray: np.ndarray, provided_mask: Optional[np.ndarray] = None) -> np.ndarray: # compute binary mask
    if provided_mask is not None:
        base = provided_mask.astype(np.uint8)
    else:
        _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)# Otsu's thresholding
        base = thresh
    mask = clean_mask(base)                                                         # clean the mask
    return mask

def normalize_tensor(image: torch.Tensor) -> torch.Tensor:                          # normalize with ImageNet stats
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)                        # ImageNet mean
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)                         # ImageNet std
    return (image - mean) / std                                                     # normalize

#### Classical texture features (154 dims)

In [127]:
def gabor_features(gray: np.ndarray, mask: np.ndarray) -> np.ndarray:
    orientations = np.linspace(0, np.pi, 5, endpoint=False)
    wavelengths = [2, 4, 8, 16]
    feats = []
    masked_gray = gray * (mask > 0)
    for theta in orientations:
        for wl in wavelengths:
            real, imag = gabor(masked_gray, frequency=1.0 / wl, theta=theta)
            mag = np.sqrt(real**2 + imag**2)
            resp = mag[mask > 0].mean() if mask.sum() > 0 else mag.mean()
            feats.append(resp)
    return np.array(feats, dtype=np.float32)


#### Dataset and loaders

In [128]:
class FossilDataset(Dataset): # custom dataset for fossil images
    def __init__(self, records: List[Dict], img_size: int = 224, augment: bool = False):
        self.records = records; self.img_size = img_size; self.augment = augment
        aug_transforms = [T.RandomHorizontalFlip(), T.RandomApply([T.RandomRotation(10)], p=0.5), T.RandomResizedCrop(img_size, scale=(0.9, 1.0), ratio=(0.9, 1.1)), T.ColorJitter(brightness=0.1, contrast=0.1)]
        self.train_tf = T.Compose([T.Resize((img_size, img_size))] + aug_transforms + [T.ToTensor()])
        self.eval_tf = T.Compose([T.Resize((img_size, img_size)), T.ToTensor()])
    def __len__(self): return len(self.records)
    def __getitem__(self, idx: int) -> Dict:
        rec = self.records[idx]
        img = cv2.cvtColor(cv2.imread(str(rec["path"])), cv2.COLOR_BGR2RGB)
        img = resize_and_pad(img, self.img_size)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = compute_mask(gray)
        if mask.sum() < 0.1 * mask.size:
            ys, xs = np.nonzero(mask)
            if len(xs) > 0:
                x1, x2, y1, y2 = xs.min(), xs.max(), ys.min(), ys.max(); mask[y1:y2, x1:x2] = 1
        texture = compute_texture_features(gray, mask)
        pil_img = to_pil_image(img)
        tensor = self.train_tf(pil_img) if self.augment else self.eval_tf(pil_img)
        tensor = normalize_tensor(tensor)
        return {"image": tensor, "label": rec["label"], "texture": torch.tensor(texture, dtype=torch.float32), "mask": torch.tensor(mask, dtype=torch.uint8), "path": str(rec["path"])}

def make_loaders(splits: Dict[str, List[Dict]], cfg: Config) -> Dict[str, DataLoader]:
    loaders = {}
    for split, augment in zip(["train", "val", "test"], [True, False, False]):
        ds = FossilDataset(splits[split], img_size=cfg.img_size, augment=augment)
        loaders[split] = DataLoader(ds, batch_size=cfg.batch_size, shuffle=augment, num_workers=cfg.num_workers, pin_memory=True)
    return loaders

#### Models

In [129]:
class ResNetClassifier(nn.Module):
    def __init__(self, num_classes: int):
        super().__init__(); self.backbone = torch.hub.load("pytorch/vision:v0.15.2", "resnet50", weights="IMAGENET1K_V2")
        feat_dim = self.backbone.fc.in_features; self.backbone.fc = nn.Identity(); self.classifier = nn.Linear(feat_dim, num_classes)
    def forward(self, x): feats = self.backbone(x); logits = self.classifier(feats); return logits, feats

class ViTClassifier(nn.Module):
    def __init__(self, num_classes: int):
        super().__init__(); self.backbone = timm.create_model("vit_small_patch16_224", pretrained=True, num_classes=0, global_pool="token")
        feat_dim = self.backbone.num_features; self.classifier = nn.Linear(feat_dim, num_classes)
    def forward(self, x): feats = self.backbone(x); logits = self.classifier(feats); return logits, feats

#### Training and evaluation

In [130]:
def run_epoch(model: nn.Module, loader: DataLoader, optimizer, device: str, train: bool = True, split: str = "train"):
    model.train(train)
    total_loss = 0.0
    all_preds, all_labels = [], []
    iterator = tqdm(loader, desc=f"{split} ({'train' if train else 'eval'})", leave=False)
    for batch in iterator:
        imgs = batch["image"].to(device)
        labels = batch["label"].to(device)
        logits, _ = model(imgs)
        loss = F.cross_entropy(logits, labels)
        if train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        total_loss += loss.item() * labels.size(0)
        preds = logits.argmax(dim=1).detach().cpu().numpy()
        all_preds.append(preds)
        all_labels.append(labels.cpu().numpy())
        iterator.set_postfix({"loss": f"{loss.item():.4f}"})
    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)
    avg_loss = total_loss / len(loader.dataset)
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average="macro")
    return avg_loss, acc, f1

def train_model(model: nn.Module, loaders: Dict[str, DataLoader], cfg: Config, tag: str) -> nn.Module:
    device = cfg.device
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=cfg.max_epochs)
    best_f1 = -1
    best_state = None
    wait = 0
    for epoch in range(cfg.max_epochs):
        train_loss, train_acc, train_f1 = run_epoch(model, loaders["train"], optimizer, device, train=True, split=f"train {epoch+1}/{cfg.max_epochs}")
        val_loss, val_acc, val_f1 = run_epoch(model, loaders["val"], optimizer, device, train=False, split=f"val {epoch+1}/{cfg.max_epochs}")
        scheduler.step()
        print(f"[{tag}] Epoch {epoch+1:03d} | train f1 {train_f1:.3f} acc {train_acc:.3f} | val f1 {val_f1:.3f} acc {val_acc:.3f}")
        if val_f1 > best_f1:
            best_f1 = val_f1
            best_state = model.state_dict()
            wait = 0
        else:
            wait += 1
        if wait >= cfg.patience:
            print(f"[{tag}] Early stop at epoch {epoch+1}")
            break
    if best_state is None:
        best_state = model.state_dict()
    model.load_state_dict(best_state)
    ensure_dir(cfg.ckptdir)
    torch.save(model.state_dict(), cfg.ckptdir / f"{tag}_best.pth")
    return model

def evaluate(model: nn.Module, loader: DataLoader, device: str) -> Dict:
    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for batch in loader:
            logits, _ = model(batch["image"].to(device))
            preds.append(logits.argmax(dim=1).cpu().numpy())
            labels.append(batch["label"].cpu().numpy())
    preds = np.concatenate(preds)
    labels = np.concatenate(labels)
    cm = confusion_matrix(labels, preds)
    return {
        "acc": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="macro"),
        "cm": cm,
        "labels": labels,
        "preds": preds,
    }

def plot_confusion(cm: np.ndarray, classes: List[str], figpath: Path, title: str):
    fig, ax = plt.subplots(figsize=(6, 5))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    disp.plot(ax=ax, cmap="Blues", colorbar=False)
    ax.set_title(title)
    fig.tight_layout()
    fig.savefig(figpath, dpi=200)
    plt.close(fig)

def plot_metric_bar(results: Dict[str, Dict], figpath: Path, metric: str = "f1"):
    names = list(results.keys())
    vals = [results[n][metric] for n in names]
    fig, ax = plt.subplots(figsize=(6, 4))
    sns.barplot(x=names, y=vals, ax=ax)
    ax.set_ylabel(metric)
    ax.set_ylim(0, 1)
    ax.set_title(f"{metric} comparison")
    fig.tight_layout()
    fig.savefig(figpath, dpi=200)
    plt.close(fig)


#### Hybrid classifiers

In [131]:
@torch.no_grad()
def collect_embeddings(model, loader, device: str): # collect deep features, texture features, labels
    model.eval(); all_feats, all_textures, all_labels = [], [], []
    for batch in loader:
        imgs = batch["image"].to(device)
        logits, feats = model(imgs)
        all_feats.append(feats.cpu().numpy())
        all_textures.append(batch["texture"].numpy())
        all_labels.append(batch["label"].numpy())
    return np.concatenate(all_feats), np.concatenate(all_textures), np.concatenate(all_labels)

def train_hybrid(feats, textures, labels):
    X_tex = textures; X_feat = feats; X_hybrid = np.concatenate([textures, feats], axis=1)
    models = {
        "texture_only": LogisticRegression(max_iter=200, n_jobs=-1, multi_class="auto").fit(X_tex, labels),
        "deep_only": LogisticRegression(max_iter=200, n_jobs=-1, multi_class="auto").fit(X_feat, labels),
        "hybrid_logreg": LogisticRegression(max_iter=200, n_jobs=-1, multi_class="auto").fit(X_hybrid, labels),
        "hybrid_mlp": MLPClassifier(hidden_layer_sizes=(256,), max_iter=300).fit(X_hybrid, labels),
    }
    return models

def eval_hybrid(models, feats, textures, labels):
    X_tex = textures; X_feat = feats; X_hybrid = np.concatenate([textures, feats], axis=1)
    results = {}
    for name, model in models.items():
        if "tex" in name:
            X = X_tex
        elif "deep" in name and "hybrid" not in name:
            X = X_feat
        else:
            X = X_hybrid
        preds = model.predict(X)
        results[name] = {"acc": accuracy_score(labels, preds), "f1": f1_score(labels, preds, average="macro")}
    return results

In [132]:
# XAI
def grad_cam_cnn(model: ResNetClassifier, img_tensor: torch.Tensor, target_class: int) -> np.ndarray:
    model.eval(); target_layer = model.backbone.layer4[-1].conv3; activations, gradients = [], []
    def fwd_hook(_, __, output): activations.append(output.detach())
    def bwd_hook(_, grad_input, grad_output): gradients.append(grad_output[0].detach())
    handle_fwd = target_layer.register_forward_hook(fwd_hook); handle_bwd = target_layer.register_full_backward_hook(bwd_hook)
    img_tensor = img_tensor.unsqueeze(0).to(next(model.parameters()).device); logits, _ = model(img_tensor); score = logits[0, target_class]
    model.zero_grad(); score.backward(); acts = activations[0]; grads = gradients[0]; weights = grads.mean(dim=(2, 3), keepdim=True)
    cam = (weights * acts).sum(dim=1); cam = F.relu(cam); cam = cam.squeeze().cpu().numpy(); cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-6)
    handle_fwd.remove(); handle_bwd.remove(); return cam

def occlusion_sensitivity(model, img_tensor: torch.Tensor, target_class: int, patch: int = 32) -> np.ndarray:
    model.eval(); device = next(model.parameters()).device; img_tensor = img_tensor.unsqueeze(0).to(device)
    def forward_fn(x): logits, _ = model(x); return logits
    occ = Occlusion(forward_fn)
    attributions = occ.attribute(img_tensor, strides=(3, patch, patch), sliding_window_shapes=(3, patch, patch), target=target_class)
    attr = attributions.squeeze().mean(dim=0).cpu().numpy(); attr = (attr - attr.min()) / (attr.max() - attr.min() + 1e-8); return attr

def overlay_cam(img: np.ndarray, cam: np.ndarray, alpha: float = 0.4) -> np.ndarray:
    cam_resized = cv2.resize(cam, (img.shape[1], img.shape[0])); heatmap = cv2.applyColorMap(np.uint8(255 * cam_resized), cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(img, 1 - alpha, heatmap, alpha, 0); return overlay

def build_concept_directions(concept_root: Path, model: nn.Module, cfg: Config, random_pool: np.ndarray) -> Dict[str, np.ndarray]:
    if not concept_root.exists():
        print(f"No concept bank at {concept_root}, skipping TCAV."); return {}
    device = cfg.device; model.eval(); transform = T.Compose([T.Resize((cfg.img_size, cfg.img_size)), T.ToTensor(), normalize_tensor]); directions = {}
    for concept_dir in concept_root.iterdir():
        if not concept_dir.is_dir():
            continue
        feat_list = []
        for img_path in concept_dir.glob("*"):
            if img_path.suffix.lower() not in [".jpg", ".png", ".jpeg"]:
                continue
            img = cv2.cvtColor(cv2.imread(str(img_path)), cv2.COLOR_BGR2RGB); img = resize_and_pad(img, cfg.img_size)
            tensor = transform(to_pil_image(img)).unsqueeze(0).to(device)
            with torch.no_grad():
                _, feats = model(tensor)
            feat_list.append(feats.cpu().numpy().squeeze())
        if not feat_list:
            continue
        feats = np.stack(feat_list)
        rand_idx = np.random.choice(len(random_pool), size=len(feats), replace=len(random_pool) < len(feats))
        rand_feats = random_pool[rand_idx]
        X = np.vstack([feats, rand_feats]); y = np.array([1] * len(feats) + [0] * len(rand_feats))
        clf = LogisticRegression(max_iter=200).fit(X, y); direction = clf.coef_.flatten(); direction = direction / (np.linalg.norm(direction) + 1e-8)
        directions[concept_dir.name] = direction
    return directions

def tcav_scores(model: nn.Module, loader: DataLoader, concept_dirs: Dict[str, np.ndarray], device: str) -> Dict[str, float]:
    if not concept_dirs:
        return {}
    model.eval(); scores = {k: [] for k in concept_dirs}
    for batch in loader:
        imgs = batch["image"].to(device); imgs.requires_grad_(True); logits, feats = model(imgs); preds = logits.argmax(dim=1)
        for i in range(imgs.size(0)):
            model.zero_grad(); logit = logits[i, preds[i]]; grad = torch.autograd.grad(logit, feats, retain_graph=True)[0][i]
            for concept, direction in concept_dirs.items():
                dir_t = torch.tensor(direction, device=device, dtype=grad.dtype)
                directional = torch.dot(grad, dir_t)
                scores[concept].append(float(directional.item() > 0))
    return {k: float(np.mean(v)) if v else 0.0 for k, v in scores.items()}

In [ ]:
# Orchestrate full run on Geo Fossils-I (runtime depends on hardware)
cfg = Config(root=Path("geo fossil I"), num_workers=0)
print("Using device:", cfg.device)
ensure_dir(cfg.figdir); ensure_dir(cfg.ckptdir)
manifest = list_images_by_class(cfg.root); classnames = sorted({r["classname"] for r in manifest}); splits = stratified_split(manifest)
with open("splits.json", "w") as f:
    json.dump({k: [str(r["path"]) for r in v] for k, v in splits.items()}, f, indent=2)
loaders = make_loaders(splits, cfg)

set_seed(cfg.seeds[0]); cnn = ResNetClassifier(num_classes=len(classnames)); cnn = train_model(cnn, loaders, cfg, tag="cnn")
cnn_test = evaluate(cnn, loaders["test"], cfg.device); plot_confusion(cnn_test["cm"], classnames, cfg.figdir / "cm_cnn.png", "CNN Confusion (Geo Fossils-I)")

set_seed(cfg.seeds[1]); vit = ViTClassifier(num_classes=len(classnames)); vit = train_model(vit, loaders, cfg, tag="vit")
vit_test = evaluate(vit, loaders["test"], cfg.device); plot_confusion(vit_test["cm"], classnames, cfg.figdir / "cm_vit.png", "ViT Confusion (Geo Fossils-I)")

feats_train, tex_train, y_train = collect_embeddings(cnn, loaders["train"], cfg.device)
feats_test, tex_test, y_test = collect_embeddings(cnn, loaders["test"], cfg.device)
hybrid_models = train_hybrid(feats_train, tex_train, y_train); hybrid_results = eval_hybrid(hybrid_models, feats_test, tex_test, y_test)
hybrid_results["cnn_only"] = {"acc": cnn_test["acc"], "f1": cnn_test["f1"]}; plot_metric_bar(hybrid_results, cfg.figdir / "hybrid_f1.png", metric="f1")

concept_dirs = build_concept_directions(cfg.concept_root, cnn, cfg, feats_train)
if concept_dirs:
    tcav = tcav_scores(cnn, loaders["test"], concept_dirs, cfg.device)
    if tcav:
        fig, ax = plt.subplots(figsize=(5, 3)); names = list(tcav.keys()); vals = [tcav[k] for k in names]
        sns.barplot(x=names, y=vals, ax=ax); ax.set_ylabel("TCAV score (fraction positive)"); ax.set_title("Concept influence (CNN)")
        fig.tight_layout(); fig.savefig(cfg.figdir / "tcav_cnn.png", dpi=200); plt.close(fig)

batch = next(iter(loaders["test"])); img_tensor = batch["image"][0]
img_np = (img_tensor.permute(1, 2, 0).numpy() * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])); img_np = np.clip(img_np * 255, 0, 255).astype(np.uint8)
cnn_logits, _ = cnn(img_tensor.unsqueeze(0).to(cfg.device)); pred_class = int(cnn_logits.argmax(dim=1).item())
cam = grad_cam_cnn(cnn, img_tensor, pred_class); cam_overlay = overlay_cam(img_np, cam); plt.imsave(cfg.figdir / "gradcam_cnn.png", cam_overlay)
occ = occlusion_sensitivity(cnn, img_tensor, pred_class, patch=28); plt.imsave(cfg.figdir / "occlusion_cnn.png", occ, cmap="inferno")

print("Saved figures:", list(cfg.figdir.glob("*.png")))
print("CNN test macro-F1:", cnn_test["f1"], "ViT test macro-F1:", vit_test["f1"])

Using device: cuda


Using cache found in C:\Users\s.swapnil/.cache\torch\hub\pytorch_vision_v0.15.2


train 1/100 (train):   0%|          | 0/1 [00:00<?, ?it/s]

## How to use the figures in your Results section
- `cm_cnn.png` / `cm_vit.png`: report per-class confusions and overall macro-F1; compare CNN vs ViT.
- `hybrid_f1.png`: shows gains from adding interpretable texture features (texture-only, deep-only, hybrid).
- `gradcam_cnn.png`: highlight salient texture motifs (spirals, ridges, reticulation) the CNN uses.
- `occlusion_cnn.png`: evidences critical patches?dark areas = drop in confidence when occluded.
- `tcav_cnn.png` (if available): concept influence scores quantifying reliance on spiral/ridge/etc. patches.

Reduced-FID (reduced-FID/): Multiple real fossil classes (e.g., ammonoid, agnatha, amphibian, angiosperm, avialae, etc.).

- Test: Zero-shot classification with your trained CNN/ViT/hybrid. Build a fixed eval split per class; report accuracy/macro-F1 and confusion matrices (e.g., cm_fid.png).
- XAI: Run Grad-CAM/occlusion on a few images per class to see which real-image textures are used.
- Texture drift: Compute the 154-dim texture vector on salient regions and compare distributions vs Geo Fossils-I (ridge spacing, GLCM contrast, roughness).
- Concept reliance: If you have a concept bank, run TCAV on FID to see if spiral/ridge/reticulate concepts still drive decisions.